# RNN起名器（二）——RNN基础

```
这篇博客介绍RNN基础。
```

具体关于RNN的细节介绍和实现，推荐wildml的[这个系列博客](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)，一共四篇，带你由浅入深学习RNN和Theano实现。补充：看到知乎上有人把这个系列翻译出来了，可以[看看](https://zhuanlan.zhihu.com/p/22266022)。

另外，Karpathy（char-rnn的作者）的[博客](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)讲了更多关于RNN和语言模型的知识。

这里我简单总结下。

## 1. RNN

这里说的RNN特指循环神经网络（Recurrent Neural Network），先看下面这张图：

![](https://raw.githubusercontent.com/applenob/nlp_projects/master/resources/rnn.jpg)

简单介绍下符号：

- $x_t$是t时刻的输入；
- $s_t$是t时刻的隐藏状态，可以理解成网络的“记忆”；
- $o_t$是t时刻的输出


先看左边没有展开的图，若输入x直接到输出o，那就是一个普通的神经元；RNN就是在原来的基础上，增加了一个中间状态s，然后这个中间状态会受到上一个时间点（序列中前一个）中间状态的影响。

按时间展开这个RNN单元，就可以获得右边的图，U，V，W是唯一的。

用数学语言描述上面的图：

$$s_t = tanh(Ux_t+Ws_{t-1})$$

$$o_t = softmax(Vs_t)$$

下面给出theano实现：

```python
# 1.先定义单个时间序列下的计算规则
def forward_prop_step(x_t, s_t_prev, U, V, W):
    s_t = T.tanh(U[:, x_t] + W.dot(s_t_prev))
    o_t = T.nnet.softmax(V.dot(s_t))
    return [o_t[0], s_t]
# 2.对输入序列x，计算隐藏状态序列s，和输出序列o；thean中的scan相当于for循环。
[o, s], updates = theano.scan(
    forward_prop_step,
    sequences=x,
    outputs_info=[None, dict(initial=T.zeros(self.hidden_dim))],
    non_sequences=[U, V, W],
    truncate_gradient=self.bptt_truncate,
    strict=True)
```

预测和损失函数：

```python
prediction = T.argmax(o, axis=1)
o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
```

## 2. GRU（门限递归单元）

普通的RNN会有**梯度消失**的问题，即对于较长的输入序列，会出现梯度为0的状况。

于是产生了GRU（门限递归单元）和LSTM（长短项记忆）。

GRU：

![](https://raw.githubusercontent.com/applenob/nlp_projects/master/resources/gru.png)

解释：

- $z = \sigma (x_tU^z + s_{t-1}W^z)$ ，z是update gate（更新门），决定保留多少之前的记忆和候选记忆；
- $r = \sigma (x_tU^r + s_{t-1}W^r)$ ，r是reset gate（重置门），决定如何结合新的输入和之前的记忆；
- $h = tanh(x_tU^h + (s_{t-1}\circ r)W^h)$ ，h是候选的s，对应图中的$\tilde h$
- $s_t = (1-z)\circ h + z \circ s_{t-1} $ 是t时刻的隐藏状态，可以理解成网络的“记忆”，对应图中的h。

Theano实现：

```python
# 获得embedding
x_e = E[:, x_t]

# GRU 
z_t = T.nnet.hard_sigmoid(U[0].dot(x_e) + W[0].dot(s_t_prev) + b[0])
r_t = T.nnet.hard_sigmoid(U[1].dot(x_e) + W[1].dot(s_t_prev) + b[1])
h_t = T.tanh(U[2].dot(x_e) + W[2].dot(s_t_prev * r_t) + b[2])
s_t = (T.ones_like(z_t) - z_t) * h_t + z_t * s_t_prev
```

## 3. LSTM（长短项记忆）

继续介绍LSTM：

![](https://raw.githubusercontent.com/applenob/nlp_projects/master/resources/lstm.png)

解释：

- $i = \sigma (x_tU^i + s_{t-1}W^i)$ ，i是input gate（输入门），定义多少候选记忆可以通过；
- $f = \sigma (x_tU^f + s_{t-1}W^i)$ ，f是forget gate（遗忘门），定义多少之前的记忆可以通过；
- $o = \sigma (x_tU^o + s_{t-1}W^o)$ ，o是output gate（输出门），定义多少内部记忆要暴露给外部网络；
- $g = tanh(x_tU^g + s_{t-1}W^g)$ ， g是候选记忆；
- $c_t = c_{t-1}\circ f + g \circ i$ ，$c_t$是内部记忆；
- $s_t = tanh(c_t) \circ o$ ，$s_t$是外部记忆，暴露给网络。

Tensorflow实现：

参数定义：

```python  
# Parameters:
  # Input gate: input, previous output, and bias.
  iU = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  iW = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fU = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fW = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cU = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cW = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  oU = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  oW = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
```

计算规则：

```python
def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, iU) + tf.matmul(o, iW) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fU) + tf.matmul(o, fW) + fb)
    update = tf.matmul(i, cU) + tf.matmul(o, cW) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, oU) + tf.matmul(o, oW) + ob)
    return output_gate * tf.tanh(state), state
```

## 4. 一些细节

符号$\circ$代表矩阵的Element-wise product，也称为Hadamard product。

在theano和tensorflow中，*直接表示Element-wise product。

而theano中的.dot和tensorflow中的tf.matmul，表示矩阵乘法。

In [1]:
print "end of part 2 "

end of part 2 
